In [19]:
import os
import numpy as np
import cv2
from sklearn.utils import shuffle
from tensorflow.keras.models import load_model
import imutils   

In [20]:
def crop_brain_contour(image, plot=False):
    
    # Convert the image to grayscale, and blur it slightly
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Find contours in thresholded image, then grab the largest one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    # extreme points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    
    # crop new image out of the original image using the four extreme points (left, right, top, bottom)
    new_image = image[extTop[1]:extBot[1], extLeft[0]:extRight[0]]            

    if plot:
        plt.figure()
        plt.subplot(1, 2, 1)
        plt.imshow(image)
        plt.tick_params(axis='both', which='both', top=False, bottom=False, left=False, right=False,labelbottom=False, labeltop=False, labelleft=False, labelright=False)
        plt.title('Original Image')
        plt.subplot(1, 2, 2)
        plt.imshow(new_image)
        plt.tick_params(axis='both', which='both',top=False, bottom=False, left=False, right=False,labelbottom=False, labeltop=False, labelleft=False, labelright=False)
        plt.title('Cropped Image')
        plt.show()
    
    return new_image

In [21]:
def load_data(dir_list, image_size):
    # load all images in a directory
    X = []
    y = []
    image_width, image_height = image_size

    for directory in dir_list:
        for filename in os.listdir(directory):
            image = cv2.imread(os.path.join(directory, filename))
            image = crop_brain_contour(image, plot=False)
            image = cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
            # normalize values
            image = image / 255.0
            # convert image to numpy array and append it to X
            X.append(image)
            # append a value of 1 to the target array if the image is in 'yes' folder, else 0.
            if directory.endswith('yes'):
                y.append(1)
            else:
                y.append(0)

    X = np.array(X)
    y = np.array(y)

    # Shuffle the data
    X, y = shuffle(X, y)

    print(f'Number of examples: {len(X)}')
    print(f'X shape: {X.shape}')
    print(f'y shape: {y.shape}')

    return X, y


In [22]:
test_dir_yes = os.getcwd() + '\\final_test_set\\yes\\'
test_dir_no = os.getcwd() + '\\final_test_set\\no\\'
test_dirs = [test_dir_yes, test_dir_no]
image_size = (240, 240)  # Image size used during training

In [23]:
X_test, y_test = load_data(test_dirs, image_size)

Number of examples: 51
X shape: (51, 240, 240, 3)
y shape: (51,)


In [24]:
model = load_model('100Rl0Gen.keras')

In [25]:
predictions = model.predict(X_test)
predicted_classes = np.where(predictions > 0.5, 1, 0)

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000001D827FB3B00> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step


In [26]:
accuracy = np.mean(predicted_classes.flatten() == y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 33.33%


In [27]:
# Optionally, display how many were correctly predicted
correct_predictions = np.sum(predicted_classes.flatten() == y_test)
total_predictions = len(y_test)
print(f"Correctly Predicted: {correct_predictions}/{total_predictions}")

Correctly Predicted: 17/51
